In [2]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=36b45b8e17b8149fab04a9b7ef1b44668241233f84d2c304ca18cd5f97f32245
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [31]:
import findspark
findspark.init()


In [32]:
from pyspark import SparkContext

In [33]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("CrimeDataAnalysis") \
    .getOrCreate()

# Now you can use the 'spark' variable to read CSV files or perform other Spark operations


In [34]:
!wget https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/

--2024-05-12 03:36:58--  https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.140.199, 52.206.68.26
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html.1’

index.html.1            [  <=>               ] 771.94K  2.16MB/s    in 0.3s    

2024-05-12 03:37:00 (2.16 MB/s) - ‘index.html.1’ saved [790469]



In [340]:
from pyspark.sql.functions import to_timestamp,col,lit
read_csv = spark.read.csv('sample_data/crime.csv', header=True).withColumn('DATE  OF OCCURRENCE', to_timestamp(col('DATE  OF OCCURRENCE'), 'MM/dd/yyyy hh:mm:ss a'))
read_csv.show(5)


+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#|DATE  OF OCCURRENCE|               BLOCK| IUCR| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|
+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|JG497095|2023-11-08 20:50:00| 025XX N KEDZIE BLVD| 0810|               THEFT|             OVER $500|               STREET|     N|       N|1414|  35|    06|     1154609|     1916759|41.927407329| -87.70729439|(41.927407329, -8...|
|JG496991|2023-11-08 15:14:00| 0000X W CHICAGO AVE| 0560|             ASSAUL

**Schema Handling**

In [341]:
read_csv.printSchema()

root
 |-- CASE#: string (nullable = true)
 |-- DATE  OF OCCURRENCE: timestamp (nullable = true)
 |-- BLOCK: string (nullable = true)
 |--  IUCR: string (nullable = true)
 |--  PRIMARY DESCRIPTION: string (nullable = true)
 |--  SECONDARY DESCRIPTION: string (nullable = true)
 |--  LOCATION DESCRIPTION: string (nullable = true)
 |-- ARREST: string (nullable = true)
 |-- DOMESTIC: string (nullable = true)
 |-- BEAT: string (nullable = true)
 |-- WARD: string (nullable = true)
 |-- FBI CD: string (nullable = true)
 |-- X COORDINATE: string (nullable = true)
 |-- Y COORDINATE: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- LOCATION: string (nullable = true)



In [342]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, DoubleType, IntegerType

In [343]:
read_csv.columns

['CASE#',
 'DATE  OF OCCURRENCE',
 'BLOCK',
 ' IUCR',
 ' PRIMARY DESCRIPTION',
 ' SECONDARY DESCRIPTION',
 ' LOCATION DESCRIPTION',
 'ARREST',
 'DOMESTIC',
 'BEAT',
 'WARD',
 'FBI CD',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION']

In [344]:
labels= [('CASE#',StringType()),
('DATE  OF OCCURRENCE',TimestampType()),
('BLOCK',StringType()),
(' IUCR',StringType()),
(' PRIMARY DESCRIPTION',StringType()),
(' SECONDARY DESCRIPTION',StringType()),
(' LOCATION DESCRIPTION',StringType()),
('ARREST',StringType()),
('DOMESTIC', BooleanType()),
('BEAT', StringType()),
('WARD',StringType()),
('FBI CD', StringType()),
('X COORDINATE',StringType()),
('Y COORDINATE',StringType()),
('LATITUDE', DoubleType()),
('LONGITUDE', DoubleType()),
('LOCATION', StringType())]

In [345]:
schema = StructType([StructField(x[0],x[1],True) for x in labels])


In [346]:
csv_df = spark.read.csv('sample_data/crime.csv', schema = schema)

In [347]:
csv_df.printSchema()

root
 |-- CASE#: string (nullable = true)
 |-- DATE  OF OCCURRENCE: timestamp (nullable = true)
 |-- BLOCK: string (nullable = true)
 |--  IUCR: string (nullable = true)
 |--  PRIMARY DESCRIPTION: string (nullable = true)
 |--  SECONDARY DESCRIPTION: string (nullable = true)
 |--  LOCATION DESCRIPTION: string (nullable = true)
 |-- ARREST: string (nullable = true)
 |-- DOMESTIC: boolean (nullable = true)
 |-- BEAT: string (nullable = true)
 |-- WARD: string (nullable = true)
 |-- FBI CD: string (nullable = true)
 |-- X COORDINATE: string (nullable = true)
 |-- Y COORDINATE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)



In [348]:
csv_df.show(5)

+--------+-------------------+-------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#|DATE  OF OCCURRENCE|              BLOCK| IUCR| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|
+--------+-------------------+-------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#|               NULL|              BLOCK| IUCR| PRIMARY DESCRIPTION|   SECONDARY DESCRI...|  LOCATION DESCRIP...|ARREST|    NULL|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|        NULL|         NULL|            LOCATION|
|JG497095|               NULL|025XX N KEDZIE BLVD| 0810|               THEFT|   

**Showing column**

In [349]:
read_csv.select(' IUCR').show(5)

+-----+
| IUCR|
+-----+
| 0810|
| 0560|
| 051A|
| 0820|
| 0810|
+-----+
only showing top 5 rows



In [350]:
read_csv.select(col(' IUCR')).show(5)

+-----+
| IUCR|
+-----+
| 0810|
| 0560|
| 051A|
| 0820|
| 0810|
+-----+
only showing top 5 rows



**Showing multiple cols**

In [351]:
read_csv.select(' IUCR', ' PRIMARY DESCRIPTION', 'DOMESTIC').show(5)

+-----+--------------------+--------+
| IUCR| PRIMARY DESCRIPTION|DOMESTIC|
+-----+--------------------+--------+
| 0810|               THEFT|       N|
| 0560|             ASSAULT|       N|
| 051A|             ASSAULT|       N|
| 0820|               THEFT|       N|
| 0810|               THEFT|       N|
+-----+--------------------+--------+
only showing top 5 rows



**Add new column named NEW COL with all entries 1s**

In [352]:
from pyspark.sql.functions import lit

In [353]:
read_csv.withColumn('NEW COLUMN', lit(1)).show(5)

+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+----------+
|   CASE#|DATE  OF OCCURRENCE|               BLOCK| IUCR| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|NEW COLUMN|
+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+----------+
|JG497095|2023-11-08 20:50:00| 025XX N KEDZIE BLVD| 0810|               THEFT|             OVER $500|               STREET|     N|       N|1414|  35|    06|     1154609|     1916759|41.927407329| -87.70729439|(41.927407329, -8...|         1|
|JG496991|2023-11-08 15:14:00| 0

**Dropping column ` IUCR`**

In [354]:
read_csv.drop(' IUCR').show(5)

+--------+-------------------+--------------------+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#|DATE  OF OCCURRENCE|               BLOCK| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|
+--------+-------------------+--------------------+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|JG497095|2023-11-08 20:50:00| 025XX N KEDZIE BLVD|               THEFT|             OVER $500|               STREET|     N|       N|1414|  35|    06|     1154609|     1916759|41.927407329| -87.70729439|(41.927407329, -8...|
|JG496991|2023-11-08 15:14:00| 0000X W CHICAGO AVE|             ASSAULT|                SIMPLE|     

#### WORKING WITH ROWS

In [355]:
read_csv.select('DATE  OF OCCURRENCE').show(5)

+-------------------+
|DATE  OF OCCURRENCE|
+-------------------+
|2023-11-08 20:50:00|
|2023-11-08 15:14:00|
|2023-11-08 22:55:00|
|2024-03-07 14:15:00|
|2024-03-07 04:53:00|
+-------------------+
only showing top 5 rows



Filtering on Date

In [356]:
one_day_crime_df = read_csv.filter(col('DATE  OF OCCURRENCE').contains('2023-11-01'))
one_day_crime_df.count()



772

In [357]:
one_day_crime_df.show(5)

+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#|DATE  OF OCCURRENCE|               BLOCK| IUCR| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|
+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|JG503603|2023-11-01 21:00:00|   024XX W FITCH AVE| 0930| MOTOR VEHICLE THEFT|  THEFT / RECOVERY ...| VEHICLE NON-COMME...|     N|       N|2411|  50|    07|     1158863|     1947386|42.011363066|-87.690818628|(42.011363066, -8...|
|JG518441|2023-11-01 07:00:00|044XX S DR MARTIN...| 1562|         SEX OFFENS

In [358]:
one_day_crime_df.count()

772

In [359]:
read_csv.count()

211996

In [360]:
new_csv = read_csv.union(one_day_crime_df)

In [361]:
new_csv.count()

212768

In [362]:
new_csv.orderBy('DATE  OF OCCURRENCE',ascending = False ).show(5)

+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#|DATE  OF OCCURRENCE|               BLOCK| IUCR| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|
+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|JH248763|2024-05-03 00:00:00|079XX S MUSKEGON AVE| 0281|CRIMINAL SEXUAL A...|        NON-AGGRAVATED|            APARTMENT|     N|       N| 422|   7|    02|     1196551|     1852808|41.750980041|-87.555306017|(41.750980041, -8...|
|JH248661|2024-05-03 00:00:00|     001XX E 43RD ST| 2820|       OTHER OFFENS

In [363]:
crime_type = read_csv.select(' PRIMARY DESCRIPTION').distinct()

In [364]:
crime_type.count()


31

In [365]:
crime_type_list = crime_type.select(' PRIMARY DESCRIPTION').collect()
crime_list = [row[' PRIMARY DESCRIPTION'] for row in crime_type_list]
print(crime_list)

['OFFENSE INVOLVING CHILDREN', 'CRIMINAL SEXUAL ASSAULT', 'STALKING', 'PUBLIC PEACE VIOLATION', 'OBSCENITY', 'ARSON', 'GAMBLING', 'CRIMINAL TRESPASS', 'ASSAULT', 'LIQUOR LAW VIOLATION', 'MOTOR VEHICLE THEFT', 'THEFT', 'BATTERY', 'ROBBERY', 'HOMICIDE', 'PUBLIC INDECENCY', 'HUMAN TRAFFICKING', 'INTIMIDATION', 'PROSTITUTION', 'DECEPTIVE PRACTICE', 'CONCEALED CARRY LICENSE VIOLATION', 'SEX OFFENSE', 'CRIMINAL DAMAGE', 'NARCOTICS', 'NON-CRIMINAL', 'OTHER OFFENSE', 'KIDNAPPING', 'BURGLARY', 'WEAPONS VIOLATION', 'INTERFERENCE WITH PUBLIC OFFICER', 'OTHER NARCOTIC VIOLATION']


**Top 10 number of reported crimes by Primary type in descending order of occurance**